In [361]:
import numpy as np
import pandas as pd
from scipy.linalg import solve

In [362]:
#LU Разложение L part
def HaussL(A):
    f=np.linalg.det(A)
    if f!=0:
        number=len(A)
        An_1=np.array([[0.0 for j in range(number)] for i in range(number)])
        for i in range (len(A)):
            for j in range(len(A)):
                An_1[i,j]=A[i,j]
        lmatr=np.diag(np.full(number,1))

        for i in range(number):
            mi=np.array([[0.0 for m in range(number)] for k in range(number)])
            mi_1=np.array([[0.0 for m in range(number)] for k in range(number)])
            for j in range(i+1,number):
                mi[j,i]=-An_1[j,i]/An_1[i,i]
                mi_1[j,i]=An_1[j,i]/An_1[i,i]
            for k in range (number):
                mi[k,k]=1
                mi_1[k,k]=1
            An_1=mi.dot(An_1)
            lmatr=lmatr.dot(mi_1)
        return lmatr

In [363]:
#LU Разложение U 
def HaussU(A):
    f=np.linalg.det(A)
    if f!=0:
        number=len(A)
        An_1=np.array([[0.0 for j in range(number)] for i in range(number)])
        for i in range (len(A)):
            for j in range(len(A)):
                An_1[i,j]=A[i,j]

        for i in range(number):
            mi=np.array([[0.0 for m in range(number)] for k in range(number)])
            for j in range(i+1,number):
                mi[j,i]=-An_1[j,i]/An_1[i,i]
            for k in range (number):
                mi[k,k]=1
            An_1=mi.dot(An_1)
        return An_1

In [364]:
def Ncond(A):
    #Спектральный критерий обусловленности
    normA=np.linalg.norm(A, ord=1)
    A_1=np.linalg.inv(A)
    normA_1=np.linalg.norm(A_1, ord=1)
    conds=normA*normA_1
#Объемный критерий (критерий Ортеги)
    number=len(A)
    condv=1
    for i in range(number):
        colsum=0
        for j in range(number):
            colsum=colsum+A[i][j]
        colsum=np.sqrt(np.sqrt(colsum**2))
        condv=condv*colsum
    detA=abs(np.linalg.det(A))
    condv=condv/detA
#Угловой критерий
    pros=[]
    for i in range(number):
        Asum=0
        A_1sum=0
        for j in range(number):
            Asum=Asum+A[i][j]*A[i][j]
            A_1sum=Asum+A_1[i][j]*A_1[i][j]
        pros.append(np.sqrt(Asum*A_1sum))
    conda=max(pros)
    
    cond=[]
    cond.append(conds)
    cond.append(condv)
    cond.append(conda)
    return cond

In [365]:
#cond для A,L,U
def Filling(A):
    U=HaussU(A)
    L=HaussL(A)
    resA=Ncond(A)
    resL=Ncond(L)
    resU=Ncond(U)
    conds=['spectral','volume','angular']
    data = {'name':[],'cond': [], 'num': []}
    df = pd.DataFrame(data)
    
    for j in range(3):
        new_row = pd.DataFrame({'name':'A','cond':conds[j], 'num': resA[j]}, index = [0])
        df = pd.concat([df, new_row], ignore_index= True)
    for j in range(3):
        new_row = pd.DataFrame({'name':'L','cond':conds[j], 'num': resL[j]}, index = [0])
        df = pd.concat([df, new_row], ignore_index= True)
    for j in range(3):
        new_row = pd.DataFrame({'name':'U','cond':conds[j], 'num': resU[j]}, index = [0])
        df = pd.concat([df, new_row], ignore_index= True)
    
    print(df)

In [366]:
#Регуляризация
def Reg(A,x0):
    data = {'alpha': [],'spectral':[],'volume': [], 'angular': [],'||x-x~||':[]}
    df = pd.DataFrame(data)
    u=a.dot(x0)
    for number in range(12,0,-1):
#16
#a*a+alphaE
        acon=np.matrix.conjugate(A)
        e=np.diag(np.full(len(A),1))
        alpha=10**(-(number))
        lp=A.dot(acon)+alpha*e
        res=Ncond(lp)

#a*u
        rp=acon.dot(u)

#
        z1=solve(lp,rp)
        f=z1-x0
        new_row = pd.DataFrame({'alpha': alpha,'spectral':res[0],'volume': res[1], 'angular': res[2],'||x-x~||':np.linalg.norm(f, ord=1)}, index = [0])
        df=pd.concat([df, new_row])
    print (df)

In [367]:
#LU test good
a = np.array([[4,2,0,0],[2,5,2,0],[0,2,6,2],[0,0,2,7]])
b=np.array([-0.393, -0.389,-0.393, -0.389])
a

array([[4, 2, 0, 0],
       [2, 5, 2, 0],
       [0, 2, 6, 2],
       [0, 0, 2, 7]])

In [368]:
U=HaussU(a)
U

array([[4. , 2. , 0. , 0. ],
       [0. , 4. , 2. , 0. ],
       [0. , 0. , 5. , 2. ],
       [0. , 0. , 0. , 6.2]])

In [369]:
L=HaussL(a)
L

array([[1. , 0. , 0. , 0. ],
       [0.5, 1. , 0. , 0. ],
       [0. , 0.5, 1. , 0. ],
       [0. , 0. , 0.4, 1. ]])

In [370]:
Filling(a)

  name      cond        num
0    A  spectral   6.048387
1    A    volume   0.140552
2    A   angular  53.013006
3    L  spectral   2.775000
4    L    volume   1.774824
5    L   angular   1.582909
6    U  spectral   3.075000
7    U    volume   0.079692
8    U   angular  38.453005


In [371]:
x0=solve(a,b)
z=solve(L.dot(U),b)
print(x0)
print(z)

[-0.08466532 -0.02716935 -0.04191129 -0.04359677]
[-0.08466532 -0.02716935 -0.04191129 -0.04359677]


In [372]:
#LU test not so good
a = np.array([[1,0.99],[0.99,0.98]])
b=np.array([-0.393, -0.389])
a

array([[1.  , 0.99],
       [0.99, 0.98]])

In [373]:
U=HaussU(a)
U

array([[ 1.0e+00,  9.9e-01],
       [ 0.0e+00, -1.0e-04]])

In [374]:
L=HaussL(a)
L

array([[1.  , 0.  ],
       [0.99, 1.  ]])

In [375]:
Filling(a)

  name      cond           num
0    A  spectral  39601.000000
1    A    volume  19799.747473
2    A   angular  13930.886724
3    L  spectral      3.960100
4    L    volume      1.410674
5    L   angular      2.429176
6    U  spectral  19900.000000
7    U    volume    141.067360
8    U   angular  13930.886724


In [376]:
x0=solve(a,b)
z=solve(L.dot(U),b)
print(x0)
print(z)

[ 0.3 -0.7]
[ 0.3 -0.7]


In [377]:
#LU test bad 
a = np.array([[1.2969, 0.8648],[0.2161,0.1441]])
b=np.array([-0.393, -0.389])
a

array([[1.2969, 0.8648],
       [0.2161, 0.1441]])

In [378]:
U=HaussU(a)
U

array([[1.29690000e+00, 8.64800000e-01],
       [0.00000000e+00, 7.71069472e-09]])

In [379]:
L=HaussL(a)
L

array([[1.        , 0.        ],
       [0.16662811, 1.        ]])

In [380]:
Filling(a)

  name      cond           num
0    A  spectral  3.270652e+08
1    A    volume  8.824083e+07
2    A   angular  1.348042e+08
3    L  spectral  1.361021e+00
4    L    volume  1.080106e+00
5    L   angular  1.443629e+00
6    U  spectral  2.803509e+08
7    U    volume  1.291054e+04
8    U   angular  1.348042e+08


In [381]:
x0=solve(a,b)
z=solve(L.dot(U),b)
print(x0)
print(z)

[ 27977590.04349676 -41956680.06523005]
[ 27977590.04349676 -41956680.06523005]


In [382]:
#hilbert n=20
number=20
a = np.array([[0.0 for j in range(number)] for i in range(number)])
for i in range (number):
    for j in range(number):
        a[i,j]=1/(i+j+1)
x0=np.array([1 for k in range(number)])

In [383]:
Reg(a,x0)

          alpha      spectral         volume       angular  ||x-x~||
0  1.000000e-12  1.655798e+13  2.734778e+190  1.446132e+11  0.008777
0  1.000000e-11  1.669483e+12  2.135565e+177  1.556514e+10  0.013230
0  1.000000e-10  1.667923e+11  5.709134e+163  1.736648e+09  0.030075
0  1.000000e-09  1.673503e+10  5.836593e+149  1.838854e+08  0.039572
0  1.000000e-08  1.673363e+09  2.342786e+135  2.002650e+07  0.095633
0  1.000000e-07  1.679814e+08  2.987459e+120  2.117360e+06  0.133940
0  1.000000e-06  1.668486e+07  1.501227e+105  2.248343e+05  0.287963
0  1.000000e-05  1.640395e+06   2.209686e+89  2.360572e+04  0.455442
0  1.000000e-04  1.581479e+05   1.163477e+73  2.463471e+03  0.903387
0  1.000000e-03  1.579985e+04   1.694776e+56  2.553345e+02  1.451735
0  1.000000e-02  1.588176e+03   7.851449e+38  2.638389e+01  3.009798
0  1.000000e-01  1.612059e+02   1.371645e+21  5.806572e+00  5.014512


In [384]:
x0=np.array([k for k in range(number)])
Reg(a,x0)

          alpha      spectral         volume       angular    ||x-x~||
0  1.000000e-12  1.655798e+13  2.734778e+190  1.446132e+11    0.886986
0  1.000000e-11  1.669483e+12  2.135565e+177  1.556514e+10    1.276005
0  1.000000e-10  1.667923e+11  5.709134e+163  1.736648e+09    2.690531
0  1.000000e-09  1.673503e+10  5.836593e+149  1.838854e+08    3.309774
0  1.000000e-08  1.673363e+09  2.342786e+135  2.002650e+07    7.022196
0  1.000000e-07  1.679814e+08  2.987459e+120  2.117360e+06    9.379138
0  1.000000e-06  1.668486e+07  1.501227e+105  2.248343e+05   16.855471
0  1.000000e-05  1.640395e+06   2.209686e+89  2.360572e+04   25.339420
0  1.000000e-04  1.581479e+05   1.163477e+73  2.463471e+03   39.935641
0  1.000000e-03  1.579985e+04   1.694776e+56  2.553345e+02   58.117552
0  1.000000e-02  1.588176e+03   7.851449e+38  2.638389e+01   87.009085
0  1.000000e-01  1.612059e+02   1.371645e+21  5.806572e+00  115.333140


In [385]:
#hilbert  n=25
number=25
a = np.array([[0.0 for j in range(number)] for i in range(number)])
for i in range (number):
    for j in range(number):
        a[i,j]=1/(i+j+1)
x0=np.array([1 for k in range(number)])
Reg(a,x0)

          alpha      spectral         volume       angular  ||x-x~||
0  1.000000e-12  1.924675e+13  7.391975e+248  1.487133e+11  0.011638
0  1.000000e-11  1.930439e+12  1.067044e+231  1.591492e+10  0.017415
0  1.000000e-10  1.907981e+11  6.663428e+212  1.698589e+09  0.033495
0  1.000000e-09  1.952613e+10  1.257452e+194  1.817868e+08  0.060420
0  1.000000e-08  1.916848e+09  9.319742e+174  1.908242e+07  0.101768
0  1.000000e-07  1.932425e+08  2.099746e+155  2.022134e+06  0.199656
0  1.000000e-06  1.907831e+07  1.660214e+135  2.101761e+05  0.319476
0  1.000000e-05  1.886254e+06  3.864156e+114  2.197008e+04  0.631298
0  1.000000e-04  1.845183e+05   2.863766e+93  2.264730e+03  1.055381
0  1.000000e-03  1.766724e+04   5.750809e+71  2.335262e+02  1.950202
0  1.000000e-02  1.793646e+03   3.432243e+49  2.395824e+01  3.650405
0  1.000000e-01  1.824189e+02   8.113033e+26  6.032629e+00  6.308056


In [386]:
x0=np.array([k for k in range(number)])

In [387]:
Reg(a,x0)

          alpha      spectral         volume       angular    ||x-x~||
0  1.000000e-12  1.924675e+13  7.391975e+248  1.487133e+11    1.498381
0  1.000000e-11  1.930439e+12  1.067044e+231  1.591492e+10    2.195514
0  1.000000e-10  1.907981e+11  6.663428e+212  1.698589e+09    3.676230
0  1.000000e-09  1.952613e+10  1.257452e+194  1.817868e+08    6.369290
0  1.000000e-08  1.916848e+09  9.319742e+174  1.908242e+07    9.308517
0  1.000000e-07  1.932425e+08  2.099746e+155  2.022134e+06   17.033163
0  1.000000e-06  1.907831e+07  1.660214e+135  2.101761e+05   23.793829
0  1.000000e-05  1.886254e+06  3.864156e+114  2.197008e+04   42.362813
0  1.000000e-04  1.845183e+05   2.863766e+93  2.264730e+03   58.842102
0  1.000000e-03  1.766724e+04   5.750809e+71  2.335262e+02   94.518352
0  1.000000e-02  1.793646e+03   3.432243e+49  2.395824e+01  131.960086
0  1.000000e-01  1.824189e+02   8.113033e+26  6.032629e+00  184.560829
